In [1]:
import pandas as pd
import numpy as np

In [2]:
import itertools
l_permutation = list(itertools.product([1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2],[1,2],[1, 2]))
df_permutation = pd.DataFrame(l_permutation,columns = ['beta','mu','p','delta','alpha','g_L','g_Z','g_Q','N_bar'])

In [3]:
class par: None
class moms: None
%run ./main.ipynb

Stored 'start' (str)
Stored 'end' (str)
Stored 'df_estimates' (DataFrame)
Stored 'start' (str)
Stored 'end' (str)
Stored 'df_estimates' (DataFrame)


,Parameter name,Symbol,1984 - 2000,2001 - 2016,Difference
0,Discount factor,$\beta$,$0.953$,$0.968$,$0.015$
1,Mark-up,$\mu$,$1.147$,$1.172$,$0.026$
2,Disaster probability,$p$,$0.030$,$0.075$,$0.045$
3,"Depreciation, pct.",$\delta$,$3.085$,$3.907$,$0.822$
4,Cobb-Douglas parameter,$\alpha$,$0.357$,$0.300$,$-0.057$
5,"Population growth, pct.",$g_L$,$0.263$,$0.682$,$0.419$
6,"TFP growth, pct.",$g_Z$,$1.678$,$0.715$,$-0.963$
7,"Technological change, pct.",$g_Q$,$0.678$,$0.411$,$-0.267$
8,Labour Supply,$\bar{N}$,$0.489$,$0.515$,$0.026$


\begin{tabular}{lcccc}
\toprule
& & & \textit{Estimates} & \\ \cmidrule(lr){3-5}
Parameter name & Symbol & 1984 - 2000 & 2001 - 2016 & Difference \\
\midrule
 Discount factor            & $\beta$   & $0.953$ & $0.968$ & $0.015$  \\
 Mark-up                    & $\mu$     & $1.147$ & $1.172$ & $0.026$  \\
 Disaster probability       & $p$       & $0.030$ & $0.075$ & $0.045$  \\
 Depreciation, pct.         & $\delta$  & $3.085$ & $3.907$ & $0.822$  \\
 Cobb-Douglas parameter     & $\alpha$  & $0.357$ & $0.300$ & $-0.057$ \\
 Population growth, pct.    & $g_L$     & $0.263$ & $0.682$ & $0.419$  \\
 TFP growth, pct.           & $g_Z$     & $1.678$ & $0.715$ & $-0.963$ \\
 Technological change, pct. & $g_Q$     & $0.678$ & $0.411$ & $-0.267$ \\
 Labour Supply              & $\bar{N}$ & $0.489$ & $0.515$ & $0.026$  \\
\bottomrule
\end{tabular}
Stored 'estimates' (DataFrame)


In [4]:
#Assumptions and settings
par.sigma = 0.5
par.theta = 12
par.b = -np.log(1-0.15)
par.bh = -np.log((2-np.exp(-par.b)))

#Define equations
def growthpop(par,moms):
    return par.g_L

def priceinvt(par,moms):
    return -par.g_Q

def ik(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta)

def EmpPop(par,moms):
    return par.N_bar

def ProfitY(par,moms):
    return (par.mu+par.alpha-1)/(par.mu)

def TFPgrowth(par,moms):
    moms.ProfitY = (par.mu+par.alpha-1)/(par.mu)
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    return par.g_T-(1-moms.ProfitY)*par.g_L-moms.ProfitY*(par.g_T+par.g_Q)

def ProfitK(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma)*MOM
    r_star = 1/beta_star - 1
    return ((par.mu + par.alpha -1)/(par.alpha))*(r_star + par.delta + par.g_Q*(1+r_star))

def RF(par,moms):
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
    beta_star = par.beta * (1+par.g_PC)**(-par.sigma) *MOM
    MOM2 = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))
    MOM3 = ((1-2*par.p)+par.p*np.exp(-par.bh*(-par.theta)) + par.p*np.exp(-par.b*(-par.theta))) 
    return MOM2/(MOM3*beta_star)-1

def PD(par,moms):
    par.sigma = 0.5
    par.theta = 12
    par.b = -np.log(1-0.15)
    par.bh = -np.log((2-np.exp(-par.b)))
    par.g_T = (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha)) - 1
    par.g_PC = (1+par.g_T)/(1+par.g_L) - 1
    MOM = ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta))     
    par.beta_star = par.beta * (1+par.g_PC)**(-par.sigma) * MOM
    return par.beta_star*(1+par.g_T)/(1-par.beta_star*(1+par.g_T))

result = [(ProfitK(par,moms),ProfitY(par,moms),RF(par,moms), growthpop(par,moms), priceinvt(par,moms),PD(par,moms), TFPgrowth(par,moms), ik(par,moms), EmpPop(par,moms))
            for par.beta in [estimates.loc["beta","1984 - 2000"],estimates.loc["beta","2001 - 2016"]]
            for par.mu in [estimates.loc["mu","1984 - 2000"],estimates.loc["mu","2001 - 2016"]]
            for par.p in [estimates.loc["p","1984 - 2000"],estimates.loc["p","2001 - 2016"]]
            for par.delta in [estimates.loc["delta","1984 - 2000"],estimates.loc["delta","2001 - 2016"]]
            for par.alpha in [estimates.loc["alpha","1984 - 2000"],estimates.loc["alpha","2001 - 2016"]]
            for par.g_L in [estimates.loc["g_L","1984 - 2000"],estimates.loc["g_L","2001 - 2016"]]
            for par.g_Z in [estimates.loc["g_Z","1984 - 2000"],estimates.loc["g_Z","2001 - 2016"]]
            for par.g_Q in [estimates.loc["g_Q","1984 - 2000"],estimates.loc["g_Q","2001 - 2016"]]
            for par.N_bar in [estimates.loc["N_bar","1984 - 2000"],estimates.loc["N_bar","2001 - 2016"]]
         ]
df = pd.DataFrame(result, columns =["ProfitK","ProfitY","RF", "growthpop","priceinvt","PD","TFPgrowth","ik","EmpPop"])

In [5]:
df = df_permutation.join(df)

In [6]:
moments = ["ProfitK","ProfitY","RF","ik","TFPgrowth","priceinvt","PD","growthpop","EmpPop"]
parameters = ["beta","mu","p","delta","alpha","g_L","g_Z","g_Q","N_bar"]

df_results = pd.DataFrame(columns=parameters)
df_results["moment"] = moments
df_results = df_results.set_index("moment",drop=True)

for mom in moments:
    for parm in parameters:
        result = df.loc[(df[parm] == 2),mom].mean()-df.loc[(df[parm] == 1),mom].mean() 
        df_results.loc[mom,parm] = result    
df_results

,beta,mu,p,delta,alpha,g_L,g_Z,g_Q,N_bar
moment,,,,,,,,,
ProfitK,-0.0243607,0.00802061,0.0111958,0.0122346,0.00684079,5.55112e-17,-0.0112604,-0.00524791,0
ProfitY,0,0.012878,0,0,-0.0491002,0,0,0,0
RF,-0.0156169,0,-0.0234014,0,-0.00111627,-2.08167e-17,-0.00722743,-0.000665627,0
ik,0,0,0,0.00822066,-0.00226449,0.00429945,-0.0146458,-0.00408563,0
TFPgrowth,0,-0.000336649,0,0,-1.77552e-05,4.9963e-05,-0.00841674,0.00035148,0
priceinvt,0,0,0,0,0,0,0,0.00266875,0
PD,15.4957,0,-7.80076,0,-1.34606,4.71614,-7.86346,-0.754311,0
growthpop,0,0,0,0,0,0.00418966,0,0,0
EmpPop,0,0,0,0,0,0,0,0,0.0258165
